In [118]:
from pathlib import Path
import pandas as pd
import numpy as np
import datetime
from typing import Callable, Union, Optional, TypeVar, List
from pydantic import BaseModel
import functools
from pandas import DataFrame
import Schemas
from datetime import date
from fastapi import HTTPException, status

In [119]:
# Schemas 
class AddStudent(BaseModel):
    course: str
    year: int
    name: str
        
class TakeAttendence(BaseModel):
    course: str
    year: int
    take_full_day: bool
    date: str
    present: List[str]

        
class Files(BaseModel):
    daily: Union[DataFrame, None]
    monthly: Union[DataFrame, None]
    daily_path: Path
    monthly_path: Path
        
    class Config:
        arbitrary_types_allowed = True

class Analysing(BaseModel):
    course: str
    year: int
    last_month: bool
    which_month: Optional[str] = None
    
class MostAbsentee(BaseModel):
    course: str
    year: int
        
class AttendenceCorrection(BaseModel):
    names: List[str]
    date: str
    percentage: float
    reason: str
    course: str
    year: int

In [120]:
# Decorators
BASE_PATH = Path("AttendenceFiles")
# BASE_PATH = Path("app/repository/AttendenceFiles")

def get_attendence_files(func: Callable):
    
    @functools.wraps(func)    
    
    def wrap(*args, **kwargs):
        
        open_files = kwargs['open_file'] if 'open_file' in kwargs else True
        open_daily = kwargs['open_daily'] if 'open_daily' in kwargs else False
    
        course, year = kwargs['request'].course, kwargs['request'].year
        daily_path = BASE_PATH/f"{course}{year}.csv"
        monthly_path = BASE_PATH/f"{course}{year}_monthly.csv"
        
        if open_files:
            daily = pd.read_csv(daily_path)
            monthly = pd.read_csv(monthly_path) if open_daily else None

            files = Files(daily=daily, monthly_path=monthly_path,
                          daily_path=daily_path, monthly=monthly)
            
            return func(*args, **kwargs, files=files)
            
        files = Files(daily_path=daily_path, monthly_path=monthly_path)
        
        return func(*args, **kwargs, files=files)
    
    return wrap
    

In [121]:
def save_files(func: Callable):
    @functools.wraps(func)
    
    def wraps(*args, **kwargs):
                
        save_monthly = kwargs['save_monthly'] if 'save_monthly' in kwargs else False
        only_monthly = kwargs['only_monthly'] if 'only_monthly' in kwargs else False
        
        files = func(*args, **kwargs)
        
        if not only_monthly:
            files.daily.to_csv(files.daily_path, index=False)
        
        if save_monthly or only_monthly:
            files.monthly.to_csv(files.monthly_path, index=False)
    return wraps

## Admit Students

In [122]:
@save_files
@get_attendence_files
def admit_students(request: AddStudent, files: Files, **kwargs):
    '''kwargs: open_files(default) = True'''
    
    full_names = files.daily['StudentsName'].to_list()+[request.name]
    files.daily = pd.DataFrame(data = full_names, columns=files.daily.columns)
    files.monthly = pd.DataFrame(data = full_names, columns=files.daily.columns)
    
    return files

In [123]:
help(admit_students)

Help on function admit_students in module __main__:

admit_students(request: __main__.AddStudent, files: __main__.Files, **kwargs)
    kwargs: open_files(default) = True



## Take Attendence

In [124]:
## Take Attendence
@save_files
@get_attendence_files
def take_attendence(request: TakeAttendence, files: Files, **kwargs):
    '''
    kwargs open_daily(default) = False, this will only open daily attendence file
    '''
    df = files.daily
    attendence = df[request.date].values if request.date in df else np.zeros(len(df))
    present_indices = df['StudentsName'][df['StudentsName'].isin(request.present)].index
    
    if request.take_full_day == True:
#         if attendence.any(): raise Exception("Attendence already taken...")
        assert not attendence.any(),\
            f"{request.date} attendence already taken"
        attendence[present_indices] = 1.
        
    else: attendence[present_indices] += .20

    files.daily[request.date] = attendence

    return files


In [125]:
help(take_attendence)

Help on function take_attendence in module __main__:

take_attendence(request: __main__.TakeAttendence, files: __main__.Files, **kwargs)
    kwargs open_daily(default) = False, this will only open daily attendence file



## Get Student Names

In [126]:
@get_attendence_files
def get_student_names(request: Schemas.set_class, files: Files):
    names = files.daily['StudentsName'].to_list()
    return names

In [127]:
help(get_student_names)

Help on function get_student_names in module __main__:

get_student_names(request: Schemas.set_class, files: __main__.Files)



## Analysing

In [128]:
def calculate_score(x: float):
    if x >= 90: return 5
    elif x >= 85 and x <= 89: return 4
    elif x >= 80 and x <=84: return 3
    elif x >=76 and x <= 79: return 2
    elif x == 75 and x< 76: return 1
    else: return 0

In [129]:
@get_attendence_files
def attendence_analysing(request: Analysing, files: Files):
    df = files.daily
    if request.last_month: 
        
        which_month = request.which_month[5:7]
        date_columns = [date for date in df.columns[1:] if date[5:7] == which_month]

        if not len(date_columns): raise HTTPException(status_code=status.HTTP_406_NOT_ACCEPTABLE,
                                                detail=f"No attendence taken in {which_month} month")
        return get_analysis(df, date_columns)
    

    date_columns = df.columns.to_list()

    if not len(date_columns): raise HTTPException(status_code=status.HTTP_406_NOT_ACCEPTABLE,
                                                  detail="No attendence taken in this month")
    
    return get_analysis(df, date_columns)

In [130]:
@get_attendence_files
def most_absentee(request: MostAbsentee, files: Files, **kwargs):
    df = files.daily
    
    date_columns = df.columns.to_list()

    if not len(date_columns): raise HTTPException(status_code=status.HTTP_406_NOT_ACCEPTABLE,
                                                  detail="No attendence taken in this month")
    analysis = get_analysis(df, date_columns)
    most_absentee = analysis[analysis["PERCENTAGE"] < 75]
    most_absentee = zip(most_absentee["FULL_NAME"], most_absentee["PERCENTAGE"])
    
    return most_absentee

In [131]:
def get_analysis(df: pd.DataFrame, date_columns: Union[pd.Index, List]):
    
    number_of_working_days = df.shape[1] - 1
    number_of_holidays = 90 - number_of_working_days
    df_working = df[date_columns]
    
    present_count = df_working.sum(axis=1)
    percentage = (present_count / number_of_working_days) * 100
    
    convert_to_90 = percentage / 90 * 100
    convert_to_90 = convert_to_90.round().to_list()
    
    students_names = df['StudentsName'].iloc[present_count.index].to_list()
    
    total_leav = np.count_nonzero(df_working==0.0, axis=1).tolist()
    
    data = {"FULL_NAME": students_names, "TOTAL_LEAVE": total_leav,
            "PERCENTAGE": convert_to_90}
    
    final_report = pd.DataFrame(data = data,
                        columns=["FULL_NAME", "TOTAL_LEAVE", "PERCENTAGE"])
    
    final_report["INTERNAL_MARK"] = final_report["PERCENTAGE"].apply(calculate_score)
    
    return final_report

## Attendence Corrections

In [138]:
@save_files
@get_attendence_files
def attendence_correction(request: AttendenceCorrection, files: Files):
    df = files.daily
    column = request.date
    percentage = request.percentage
    
    if column not in df.columns:
        raise HTTPException(406, detail=
                            "In this date there is no attendence taken")
    
    for name in request.names:
        index_row = df[df['StudentsName'] == name].index[0]
        current_percent = df.at[index_row, column]
        
        if current_percent <= 1.0 and (value:=current_percent+percentage) <= 1.0:
            df.at[index_row, column] = value
        else:
            df.at[index_row, column] = 1.0
            
    return files

# testing workings

In [19]:
add_student1 = AddStudent(course='bca', year=1, name='Lalkrishna')
add_student2 = AddStudent(course='bca', year=1, name='Arjun')

In [20]:
admit_students(request=add_student1)
admit_students(request=add_student2)

In [31]:
dateq = str(date(2020, 2, 4))
attnhr1 = TakeAttendence(take_full_day=False, date=dateq,\
                        present=['Arjun'], course='bca',\
                        year=1)
attnhr2 = TakeAttendence(take_full_day=True, date=dateq, present=['Arjun'],
                        course='bca',\
                        year=1)

In [30]:
take_attendence(request=attnhr1, open_daily=True)

In [33]:
take_attendence(request=attnhr2, open_daily=True)

Exception: Attendence already taken...

In [326]:
# Analysis testing

In [19]:
sample = Analysing(course='bcomfinance', year=1, last_month=False)

data = attendence_analysing(request=sample)
data

C:\Users\HP\AppData\Local\Temp/ipykernel_19248/1736518281.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  present_count = df_working.sum(axis=1)


,FULL_NAME,TOTAL_LEAVE,PERCENTAGE,INTERNAL_MARK
0,ADFADFAS,1,81.0,3
1,alkfjladfkas,1,64.0,0


In [20]:
sample = Analysing(course='bcomfinance', year=1, last_month=True, which_month='2021-08-555')

data = attendence_analysing(request=sample)
data

,FULL_NAME,TOTAL_LEAVE,PERCENTAGE,INTERNAL_MARK
0,ADFADFAS,0,67.0,0
1,alkfjladfkas,0,61.0,0


In [94]:
data.values

array([['ADFADFAS', 0, 67.0, 0],
       ['alkfjladfkas', 0, 61.0, 0]], dtype=object)

In [73]:
# Testing Most Absentees

In [74]:
sample = MostAbsentee(course='bcomfinance', year=1)

ma = most_absentee(request=sample)

for name, percentage in ma:
    print(f"Name is {name}, Percenteage is {percentage}")

Name is alkfjladfkas, Percenteage is 64.0


C:\Users\HP\AppData\Local\Temp/ipykernel_19248/1736518281.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  present_count = df_working.sum(axis=1)


In [152]:
# Testing Attendence Correction

df = pd.read_csv("AttendenceFiles/bca1.csv")
df.head(4)

,StudentsName,2021-08-15,2021-08-16,2021-08-17,2021-08-18,2021-08-19,2021-09-01,2021-09-02,2021-09-03
0,LALKRISHNA ARJUN,1.0,1.0,1.0,1.0,0.2,0.2,1.0,0.4
1,SHAHIL,0.8,1.0,1.0,1.0,1.0,0.2,0.0,0.2
2,NADIR,1.0,1.0,1.0,1.0,0.2,0.2,1.0,1.0
3,RASHITHA,1.0,1.0,1.0,1.0,1.0,0.2,0.2,1.0


In [153]:
sample_data = AttendenceCorrection(names=['LALKRISHNA ARJUN', 'NADIR'],
                                    date="2021-08-19", percentage=0.2,
                                    reason="Testing", course="bca", year=1)

attendence_correction(request=sample_data)

In [154]:
df = pd.read_csv("AttendenceFiles/bca1.csv")
df.head(4)

,StudentsName,2021-08-15,2021-08-16,2021-08-17,2021-08-18,2021-08-19,2021-09-01,2021-09-02,2021-09-03
0,LALKRISHNA ARJUN,1.0,1.0,1.0,1.0,0.4,0.2,1.0,0.4
1,SHAHIL,0.8,1.0,1.0,1.0,1.0,0.2,0.0,0.2
2,NADIR,1.0,1.0,1.0,1.0,0.4,0.2,1.0,1.0
3,RASHITHA,1.0,1.0,1.0,1.0,1.0,0.2,0.2,1.0
